In [1]:
import pandas as pd 
import numpy as np
import math

tmp = pd.read_csv('Weed_Resistance_Data.csv', sep = ',', header=None)
params_in = list(tmp[:21][0])


params_in = np.array(params_in)

In [2]:
tmp[0:23]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,5.000000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.500000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.000000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.000000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.000000e-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
params_in

array([5.0e+01, 2.5e+01, 2.0e+00, 4.0e+00, 1.0e+06, 0.0e+00, 0.0e+00,
       1.0e+01, 1.0e+00, 1.0e-08, 1.0e-08, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 0.0e+00, 5.0e-02, 5.0e-01, 9.9e-01, 5.0e-02])

In [4]:
class General:
    def __init__(self,params_in):
        self.nsim = params_in[0]             #number of simulations
        self.nYears = params_in[1]               #number of years
        self.nCohorts = params_in[2]             #number of cohorts
        self.nLoci = params_in[3]                            #number of gene loci
        self.nGeno = math.pow(3,self.nLoci)             #number of genotypes
        self.fieldSize = params_in[4]           #field size
        self.selfingCoeff = params_in[5]         #selfing coefficient 
        self.seedDelay = params_in[6]          #seed delay
        self.femaleFrac = params_in[7]
        self.f_RO = params_in[18]          #latent time
        self.f_CR = params_in[19]     #critical response
        self.f_TR = params_in[20]
        self.currentYear = 1
        self.currentCohort = 1
        self.TillingFreq = np.loadtxt('Corteva/BMP_Matlab_Code/Tilling_Data.txt')

In [5]:
class Upper:
    pass
class Lower:
    pass

In [6]:
class Init:
    def __init__(self, params_in):
        self.Upper = Upper
        self.Lower = Lower
        self.Upper.seedDensity = params_in[8]      #initial upper bank density
        self.Lower.seedDensity = params_in[9]       #initial lower bank density
        self.Upper.ResAlleleFreq = params_in[10:17:2]   #initial upper resistant allele frequency
        self.Lower.ResAlleleFreq = params_in[11:18:2] 

In [7]:
class Germination:
    def __init__(self, tmp):
        self.general = General(params_in)
        self.data = np.array(tmp[20:])
        self.survivalFraction = self.survivalfrac()


    def survivalfrac(self):
        row = 0
        survivalFraction = np.zeros((int(self.general.nYears),int(self.general.nCohorts)))
        for i in range(int(self.general.nYears)):
            for j in range(int(self.general.nCohorts)):
                survivalFraction[i][j] = self.data[row,0]
                if row < 49:
                    row += 1

        return survivalFraction
        

In [8]:
ger = Germination(tmp)
ger.survivalfrac()

array([[0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05]])

In [9]:
class Cultivation:
    def __init__(self, tmp):
        self.general = General(params_in)
        self.data = np.array(tmp[20:])
        self.survivalFraction = self.survivalfrac()


    def survivalfrac(self):
        row = 0
        survivalFraction = np.zeros((int(self.general.nYears),int(self.general.nCohorts)))
        for i in range(int(self.general.nYears)):
            for j in range(int(self.general.nCohorts)):
                survivalFraction[i][j] = self.data[row,1]
                if row < 49:
                    row += 1

        return survivalFraction

In [10]:
class Hand:
    def __init__(self, tmp):
        self.general = General(params_in)
        self.data = np.array(tmp[20:])
        self.survivalFraction = self.survivalfrac()


    def survivalfrac(self):
        row = 0
        survivalFraction = np.zeros((int(self.general.nYears),int(self.general.nCohorts)))
        for i in range(int(self.general.nYears)):
            for j in range(int(self.general.nCohorts)):
                survivalFraction[i][j] = self.data[row,2]
                if row < 49:
                    row += 1

        return survivalFraction

In [11]:
class Mature:
    def __init__(self, tmp):
        self.general = General(params_in)
        self.data = np.array(tmp[20:])
        self.maxplants = self.max_crop(3)
        self.cropParam = self.max_crop(4)
        self.weedCompetition = self.max_crop(5)

    def max_crop(self,column):
        row = 0
        survivalFraction = np.zeros((int(self.general.nYears),int(self.general.nCohorts)))
        for i in range(int(self.general.nYears)):
            for j in range(int(self.general.nCohorts)):
                survivalFraction[i][j] = self.data[row,column]
                if row < 49:
                    row += 1

        return survivalFraction

In [12]:
class SeedProd:
    def __init__(self, tmp):
        self.general = General(params_in)
        self.data = np.array(tmp[20:])
        self.maxyield = self.max_crop(6)
        self.cropParam = self.max_crop(7)
        self.weedCompetition = self.max_crop(8)

    def max_crop(self,column):
        row = 0
        survivalFraction = np.zeros((int(self.general.nYears),int(self.general.nCohorts)))
        for i in range(int(self.general.nYears)):
            for j in range(int(self.general.nCohorts)):
                survivalFraction[i][j] = self.data[row,column]
                if row < 49:
                    row += 1

        return survivalFraction

In [13]:
class Mutation:
    def __init__(self, tmp):
        self.general = General(params_in)
        self.data = np.array(tmp[20:])

        self.muR2S_locus1 = self.mut(9)
        self.muS2R_locus1 = self.mut(10)
        self.muR2S_locus2 = self.mut(11)
        self.muS2R_locus2 = self.mut(12)
        self.muR2S_locus3 = self.mut(13)
        self.muS2R_locus3 = self.mut(14)
        self.muR2S_locus4 = self.mut(15)
        self.muS2R_locus4 = self.mut(16)

    def mut(self,column):
        row = 0
        survivalFraction = np.zeros((int(self.general.nYears),int(self.general.nCohorts)))
        for i in range(int(self.general.nYears)):
            for j in range(int(self.general.nCohorts)):
                survivalFraction[i][j] = self.data[row,column]
                if row < 49:
                    row += 1

        return survivalFraction

In [14]:
class Predation:
    def __init__(self, tmp):
        self.general = General(params_in)
        self.data = np.array(tmp[20:])
        self.survivalFraction = self.survivalfrac(17)


    def survivalfrac(self,column):
        row = 0
        survivalFraction = np.zeros((int(self.general.nYears),int(self.general.nCohorts)))
        for i in range(int(self.general.nYears)):
            for j in range(int(self.general.nCohorts)):
                survivalFraction[i][j] = self.data[row,column]
                if row < 49:
                    row += 1

        return survivalFraction

In [15]:
class Winter:
    def __init__(self, tmp):
        self.general = General(params_in)
        self.data = np.array(tmp[20:])
        self.uppersurvivalFraction = self.survivalfrac(18)
        self.lowersurvivalFraction = self.survivalfrac(19)


    def survivalfrac(self,column):
        row = 0
        survivalFraction = np.zeros((int(self.general.nYears),int(self.general.nCohorts)))
        for i in range(int(self.general.nYears)):
            for j in range(int(self.general.nCohorts)):
                survivalFraction[i][j] = self.data[row,column]
                if row < 49:
                    row += 1

        return survivalFraction

In [16]:
General_TillingFrequency = np.loadtxt('Corteva/BMP_Matlab_Code/Tilling_Data.txt')
Herbicide_data = np.loadtxt('Corteva/BMP_Matlab_Code/Herbicide_Data.txt')

In [17]:
class Herbicide1:
    def __init__(self, Herbicide_data):
        self.general = General(params_in)
        self.data = Herbicide_data
        self.targetLocus = 1
        self.efficacy_SS = np.zeros((25,2))

        self.efficacy_SS[0:25,0] = Herbicide_data[0:25,1]
        self.efficacy_SS[0:25,1] = Herbicide_data[25:50,1]

        self.efficacy_RS = np.zeros((25,2))

        self.efficacy_RS[0:25,0] = Herbicide_data[0:25,2]
        self.efficacy_RS[0:25,1] = Herbicide_data[25:50,2]

        self.efficacy_RR = np.zeros((25,2))

        self.efficacy_RR[0:25,0] = Herbicide_data[0:25,3]
        self.efficacy_RR[0:25,1] = Herbicide_data[25:50,3]

In [18]:
class Herbicide2:
    def __init__(self, Herbicide_data):
        self.general = General(params_in)
        self.data = Herbicide_data
        self.targetLocus = 2
        self.efficacy_SS = np.zeros((25,2))

        self.efficacy_SS[0:25,0] = Herbicide_data[50:75,1]
        self.efficacy_SS[0:25,1] = Herbicide_data[75:100,1]

        self.efficacy_RS = np.zeros((25,2))

        self.efficacy_RS[0:25,0] = Herbicide_data[50:75,2]
        self.efficacy_RS[0:25,1] = Herbicide_data[75:100,2]

        self.efficacy_RR = np.zeros((25,2))

        self.efficacy_RR[0:25,0] = Herbicide_data[50:75,3]
        self.efficacy_RR[0:25,1] = Herbicide_data[75:100,3]

In [19]:
class Herbicide3:
    def __init__(self, Herbicide_data):
        self.general = General(params_in)
        self.data = Herbicide_data
        self.targetLocus = 3
        self.efficacy_SS = np.zeros((25,2))

        self.efficacy_SS[0:25,0] = Herbicide_data[100:125,1]
        self.efficacy_SS[0:25,1] = Herbicide_data[125:150,1]

        self.efficacy_RS = np.zeros((25,2))

        self.efficacy_RS[0:25,0] = Herbicide_data[100:125,2]
        self.efficacy_RS[0:25,1] = Herbicide_data[125:150,2]

        self.efficacy_RR = np.zeros((25,2))

        self.efficacy_RR[0:25,0] = Herbicide_data[100:125,3]
        self.efficacy_RR[0:25,1] = Herbicide_data[125:150,3]

In [20]:
class Herbicide4:
    def __init__(self, Herbicide_data):
        self.general = General(params_in)
        self.data = Herbicide_data
        self.targetLocus = 4
        self.efficacy_SS = np.zeros((25,2))

        self.efficacy_SS[0:25,0] = Herbicide_data[150:175,1]
        self.efficacy_SS[0:25,1] = Herbicide_data[175:200,1]

        self.efficacy_RS = np.zeros((25,2))

        self.efficacy_RS[0:25,0] = Herbicide_data[150:175,2]
        self.efficacy_RS[0:25,1] = Herbicide_data[175:200,2]

        self.efficacy_RR = np.zeros((25,2))

        self.efficacy_RR[0:25,0] = Herbicide_data[150:175,3]
        self.efficacy_RR[0:25,1] = Herbicide_data[175:200,3]

In [21]:
class P:
    def __init__(self,params_in, Herbicide_data,tmp):
        self.General = General(params_in)
        self.Init = Init(params_in)
        self.Germination = Germination(tmp)
        self.Cultivation = Cultivation(tmp)
        self.Hand = Hand(tmp)
        self.Mature = Mature(tmp)
        self.SeedProd = SeedProd(tmp)
        self.Mutation = Mutation(tmp)
        self.Predation = Predation(tmp)
        self.Winter = Winter(tmp)
        self.Herbicide1 = Herbicide1(Herbicide_data)
        self.Herbicide2 = Herbicide2(Herbicide_data)
        self.Herbicide3 = Herbicide3(Herbicide_data)
        self.Herbicide4 = Herbicide4(Herbicide_data)


In [22]:
par = P(params_in,Herbicide_data,tmp)

In [23]:
par.General.TillingFreq

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [24]:
class PrepareParams(P):
    def __init__(self):
        self.General = General(params_in)
        self.Init = Init(params_in)
        self.Germination = Germination(tmp)
        self.Cultivation = Cultivation(tmp)
        self.Hand = Hand(tmp)
        self.Mature = Mature(tmp)
        self.SeedProd = SeedProd(tmp)
        self.Mutation = Mutation(tmp)
        self.Predation = Predation(tmp)
        self.Winter = Winter(tmp)
        self.Herbicide1 = Herbicide1(Herbicide_data)
        self.Herbicide2 = Herbicide2(Herbicide_data)
        self.Herbicide3 = Herbicide3(Herbicide_data)
        self.Herbicide4 = Herbicide4(Herbicide_data)
        self.nLoci = int(self.General.nLoci)
        self.nGeno = int(self.General.nGeno)
        self.key, self.key_text = self.generate_key()
        self.General.UpperSeedBank = self.matingEquilibrium(self.Init.Upper.ResAlleleFreq,self.key)*self.Init.Upper.seedDensity
        self.General.LowerSeedBank = self.matingEquilibrium(self.Init.Lower.ResAlleleFreq,self.key)*self.Init.Lower.seedDensity          

        
    def generate_key(self):
        key = np.zeros((self.nLoci,self.nGeno))
        tmp = np.zeros((self.nLoci,1))

        for k in range(int(self.nGeno)):
            key[:,k] = tmp.reshape((4,))
            tmp = self.odometer(tmp,2)

        key_text = 0

        key_text = []

        for k in range(int(self.nGeno)):
            tmp = ''
            for j in range(int(self.nLoci)):
                locus_string = str(j+1)
                if key[j,k] == 0:
                    tmp = tmp + 'S' + locus_string + 'S' + locus_string
                elif key[j,k] == 1:
                    tmp = tmp + 'R' + locus_string + 'S' + locus_string
                else:
                    tmp = tmp + 'R' + locus_string + 'R' + locus_string
            key_text.append(tmp)
        # print('key text', key_text)
        return key, key_text
    
    def odometer(self,odo,base_num):
        Ndigits = len(odo)
        j = 0
        while j < Ndigits:
            if odo[j] < base_num:
                odo[j] = odo[j] +1
                break
            odo[j] = 0
            j = j+1
        return odo


    def matingEquilibrium(self,p1,key):


        q = 1 - p1


        populationFraction_outCross = np.ones((1,int(self.nGeno)))

        for j in range(int(self.nLoci)):
            for k in range((self.nGeno)):
                if key[j,k] == 2:
                    populationFraction_outCross[0][k] = populationFraction_outCross[0][k] *(math.pow(p1[j],2))
                elif key[j,k] == 1:
                    populationFraction_outCross[0][k] = populationFraction_outCross[0][k] *2*p1[j]*q[j]
                else:
                    populationFraction_outCross[0][k] = populationFraction_outCross[0][k] *(math.pow(q[j],2))

        return populationFraction_outCross[0]

In [25]:
Params = PrepareParams()

In [26]:
class Population:
    pass

In [27]:
nCohorts = int(Params.General.nCohorts)      
nYears = int(Params.General.nYears)        
nLoci = int(Params.General.nLoci)               
nGeno = int(Params.General.nGeno)              
fieldSize = int(Params.General.fieldSize)     


Population.seedBank = np.zeros((nYears,nGeno,nCohorts))


Population.lowerBank = np.zeros((nYears,nGeno))


Population.immSeed = np.zeros((nYears,nGeno,nCohorts))


Population.emSeed = np.zeros((nYears,nGeno,nCohorts))

Population.germination = np.zeros((nYears,nGeno,nCohorts))


Population.ungerminated = np.zeros((nYears,nGeno,nCohorts))


Population.herbicide1 = np.zeros((nYears,nGeno,nCohorts))
Population.herbicide2 = np.zeros((nYears,nGeno,nCohorts))
Population.herbicide3 = np.zeros((nYears,nGeno,nCohorts))
Population.herbicide4 = np.zeros((nYears,nGeno,nCohorts))


Population.mature = np.zeros((nYears,nGeno,nCohorts))


Population.hand = np.zeros((nYears,nGeno,nCohorts))


Population.immPol = np.zeros((nYears,nGeno,nCohorts))


Population.emPol = np.zeros((nYears,nGeno,nCohorts))


Population.newSeed = np.zeros((nYears,nGeno,nCohorts))


Population.mutated = np.zeros((nYears,nGeno,nCohorts))


Population.predation = np.zeros((nYears,nGeno,nCohorts))


Population.winter = np.zeros((nYears,nGeno))


Population.Res = np.zeros((1, 3))


Population.seedBank[0,:,0] = Params.General.UpperSeedBank
Population.lowerBank[0,:] = Params.General.LowerSeedBank

In [28]:
def deepTill(upperbank,lowerbank,tillFrequency,currentYear):
    if tillFrequency[currentYear] ==1:
        tmp = upperbank
        upperbank = lowerbank
        lowerbank = tmp 
    else:
        pass
    return upperbank, lowerbank





def seedImmigration(nGeno):
    seedIN = np.zeros((1,int(nGeno)))
    return seedIN






def extinction(oldPopulation,fieldSize):
    oldPopulation = oldPopulation.reshape((oldPopulation.shape[1]))

    if fieldSize == math.inf:
        newPopulation = oldPopulation
    elif fieldSize == 0:
        newPopulation = oldPopulation
    else:
        nGeno = int(len(oldPopulation))
        rndExtinct = np.random.rand(nGeno)
        newPopulation = np.zeros((nGeno))

        for i in range(nGeno):
            if oldPopulation[i] == 0:
                newPopulation[i] =0
            elif oldPopulation[i] >= 1/fieldSize:
                newPopulation[i] = oldPopulation[i]
            elif rndExtinct[i] > 0.5:
                newPopulation[i] = 1/fieldSize
            else:
                newPopulation[i] = 0

    return newPopulation.reshape((1,newPopulation.shape[0]))


def survival(oldPopulation,fSurvival):
    newPopulation = fSurvival*oldPopulation
    return newPopulation.reshape((1,oldPopulation.shape[1]))


def selectiveSurvival(oldPop,fSurvival,targetLocus,key):
    nLoci, nGeno = key.shape
    newPop = oldPop*0
    if targetLocus > nLoci:
        newPop = oldPop
    else:
        for i in range(int(nGeno)):
            if key[targetLocus-1][i] == 0:
                newPop[0][i] = fSurvival[0]*oldPop[0][i]
            elif key[targetLocus-1][i] == 1:
                newPop[0][i] = fSurvival[1]*oldPop[0][i]
            else: 
                newPop[0][i] = fSurvival[2]*oldPop[0][i]
    return newPop.reshape((1,newPop.shape[1]))


In [29]:
y = 0
c = 0
Population.seedBank[y,:,1],Population.lowerBank[y,:] = deepTill(Population.seedBank[y,:,1],
                                                                Population.lowerBank[y,:],
                                                                Params.General.TillingFreq,y)

ImmSeed = seedImmigration(nGeno)


seedBank = Population.seedBank[y,:,c].reshape(1,81) + ImmSeed

seedBank = extinction(seedBank,fieldSize)

fGerm = Params.Germination.survivalFraction[y,c]
print("seedbank shape: ", seedBank.shape)

germination = survival(seedBank,fGerm)
germination


seedbank shape:  (1, 81)


array([[0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ]])

In [30]:
ungerminated = seedBank - germination
germination = extinction(germination,fieldSize)
germination

array([[0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ]])

In [31]:
fCult = Params.Cultivation.survivalFraction[y,c]
cult = survival(germination,fCult)
cult

array([[0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ]])

In [32]:
cult = extinction(cult,fieldSize)
cult

array([[0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ]])

In [33]:
cult[0]

array([0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  ])

In [34]:
# fSS = 1 - Params.Herbicide4.efficacy_SS[y,c]
# fRS = 1 - Params.Herbicide4.efficacy_RS[y,c]
# fRR = 1 - Params.Herbicide4.efficacy_RR[y,c]


# targetLocus = Params.Herbicide4.targetLocus
# key = Params.key
# oldPop = herbicide3
# fSurvival = [fSS,fRS,fRR]


# nLoci, nGeno = key.shape
# newPop = oldPop*0
# if targetLocus > nLoci:
#     newPop = oldPop
# else:
#     for i in range(int(nGeno)):
#         if key[targetLocus][i] == 0:
#             newPop[0][i] = fSurvival[0]*oldPop[0][i]
#         elif key[targetLocus][i] == 1:
#             newPop[0][i] = fSurvival[1]*oldPop[0][i]
#         else: 
#             newPop[0] [i]= fSurvival[2]*oldPop[0][i]

In [35]:
fSS = 1 - Params.Herbicide1.efficacy_SS[y,c]
fRS = 1 - Params.Herbicide1.efficacy_RS[y,c]
fRR = 1 - Params.Herbicide1.efficacy_RR[y,c]


targetLocus = Params.Herbicide1.targetLocus



herbicide1_ = selectiveSurvival(cult,[fSS,fRS,fRR],targetLocus,Params.key)
herbicide1 = extinction(herbicide1_,fieldSize)

fSS = 1 - Params.Herbicide2.efficacy_SS[y,c]
fRS = 1 - Params.Herbicide2.efficacy_RS[y,c]
fRR = 1 - Params.Herbicide2.efficacy_RR[y,c]


targetLocus = Params.Herbicide2.targetLocus


herbicide2_ = selectiveSurvival(herbicide1,[fSS,fRS,fRR],targetLocus,Params.key)
herbicide2 = extinction(herbicide2_,fieldSize)


fSS = 1 - Params.Herbicide3.efficacy_SS[y,c]
fRS = 1 - Params.Herbicide3.efficacy_RS[y,c]
fRR = 1 - Params.Herbicide3.efficacy_RR[y,c]


targetLocus = Params.Herbicide3.targetLocus


herbicide3_ = selectiveSurvival(herbicide2,[fSS,fRS,fRR],targetLocus,Params.key)
herbicide3 = extinction(herbicide3_,fieldSize)

fSS = 1 - Params.Herbicide4.efficacy_SS[y,c]
fRS = 1 - Params.Herbicide4.efficacy_RS[y,c]
fRR = 1 - Params.Herbicide4.efficacy_RR[y,c]


targetLocus = Params.Herbicide4.targetLocus


herbicide4_ = selectiveSurvival(herbicide3,[fSS,fRS,fRR],targetLocus,Params.key)
herbicide4 = extinction(herbicide4_,fieldSize)

In [39]:
A = Params.Mature.maxplants[y,c]
B = Params.Mature.cropParam[y,c]
C = Params.Mature.weedCompetition[y,c]

In [40]:
def competition(oldPop,A,B,C):

    totalPop = sum(oldPop)
    
    a = A*oldPop
    b = (1+B+C*totalPop)


    newPop = a/b 
    return newPop

In [41]:
mature = competition(herbicide4,A,B,C)

In [43]:
mature = extinction(mature,fieldSize)

In [44]:
fHand = Params.Hand.survivalFraction[y,c]

In [46]:
hand = survival(mature,fHand)

In [47]:
A = Params.SeedProd.maxyield[y,c]
B = Params.SeedProd.cropParam[y,c]
C = Params.SeedProd.weedCompetition[y,c]

In [48]:
seedYield = competition(sum(hand),A,B,C)

In [49]:
seedYield

array([10.43362209,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [ ]:
General_nSim = params_in[0]                 #number of simulations
General_nYears = params_in[1]               #number of years
General_nCohorts = params_in[2]             #number of cohorts
General_nLoci = 4                            #number of gene loci
General_nGeno = 3^General_nLoci             #number of genotypes
General_fieldSize = params_in[3]           #field size
General_selfingCoeff = params_in[4]         #selfing coefficient 
General_seedDelay = params_in[5]          #seed delay
General_femaleFrac = params_in[6]         #fraction female
Init_Upper_seedDensity = params_in[7]      #initial upper bank density
Init_Lower_seedDensity = params_in[8]       #initial lower bank density

Init_Upper_ResAlleleFreq = params_in[9:17:2]   #initial upper resistant allele frequency
Init_Lower_ResAlleleFreq = params_in[10:18:2]  #initial lower resistant allele frequency

General_f_RO = params_in[17]          #latent time
General_f_CR = params_in[18]     #critical response
General_f_TR = params_in[19]     #complete resistance time
General_nYears = int(General_nYears)
General_nCohorts = int(General_nCohorts)


Germination_survivalFraction = np.zeros((General_nYears,General_nCohorts))
Cultivation_survivalFraction = np.zeros((General_nYears,General_nCohorts))
Hand_survivalFraction= np.zeros((General_nYears,General_nCohorts))
Mature_maxPlants= np.zeros((General_nYears,General_nCohorts))
Mature_cropParam= np.zeros((General_nYears,General_nCohorts))
Mature_weedCompetition= np.zeros((General_nYears,General_nCohorts))
SeedProd_maxYield= np.zeros((General_nYears,General_nCohorts))
SeedProd_cropParam= np.zeros((General_nYears,General_nCohorts))
SeedProd_weedCompetition= np.zeros((General_nYears,General_nCohorts))
Mutation_muR2S_locus1= np.zeros((General_nYears,General_nCohorts))
Mutation_muS2R_locus1= np.zeros((General_nYears,General_nCohorts))
Mutation_muR2S_locus2= np.zeros((General_nYears,General_nCohorts))
Mutation_muS2R_locus2= np.zeros((General_nYears,General_nCohorts))
Mutation_muR2S_locus3= np.zeros((General_nYears,General_nCohorts))
Mutation_muS2R_locus3= np.zeros((General_nYears,General_nCohorts))
Mutation_muR2S_locus4= np.zeros((General_nYears,General_nCohorts))
Mutation_muS2R_locus4= np.zeros((General_nYears,General_nCohorts))
Predation_survivalFraction= np.zeros((General_nYears,General_nCohorts))
Winter_uppersurvivalFraction= np.zeros((General_nYears,General_nCohorts))
Winter_lowersurvivalFraction= np.zeros((General_nYears,General_nCohorts))
data = np.array(tmp[20:])

In [ ]:
row = 0
for i in range(int(General_nYears)):
    for j in range(int(General_nCohorts)):
        Germination_survivalFraction[i][j] = data[row,0]
        Cultivation_survivalFraction[i][j] = data[row,1]
        Hand_survivalFraction[i][j]= data[row,2]
        Mature_maxPlants[i][j]= data[row,3]
        Mature_cropParam[i][j]= data[row,4]
        Mature_weedCompetition[i][j]= data[row,5]
        SeedProd_maxYield[i][j]= data[row,6]
        SeedProd_cropParam[i][j]= data[row,7]
        SeedProd_weedCompetition[i][j]= data[row,8]
        Mutation_muR2S_locus1[i][j]= data[row,9]
        Mutation_muS2R_locus1[i][j]= data[row,10]
        Mutation_muR2S_locus2[i][j]= data[row,11]
        Mutation_muS2R_locus2[i][j]= data[row,12]
        Mutation_muR2S_locus3[i][j]= data[row,13]
        Mutation_muS2R_locus3[i][j]= data[row,14]
        Mutation_muR2S_locus4[i][j]= data[row,15]
        Mutation_muS2R_locus4[i][j]= data[row,16]
        Predation_survivalFraction[i][j]= data[row,17]
        Winter_uppersurvivalFraction[i][j]= data[row,18]
        Winter_lowersurvivalFraction[i][j]= data[row,19]


        if row < 49:
            row += 1



In [ ]:
General_TillingFrequency = np.loadtxt('Corteva/BMP_Matlab_Code/Tilling_Data.txt')
Herbicide_data = np.loadtxt('Corteva/BMP_Matlab_Code/Herbicide_Data.txt')

In [ ]:
Herbicide_1_SS = np.zeros((25,2))

Herbicide_1_SS[0:25,0] = Herbicide_data[0:25,1]
Herbicide_1_SS[0:25,1] = Herbicide_data[25:50,1]

Herbicide_1_RS = np.zeros((25,2))

Herbicide_1_RS[0:25,0] = Herbicide_data[0:25,2]
Herbicide_1_RS[0:25,1] = Herbicide_data[25:50,2]

Herbicide_1_RR = np.zeros((25,2))

Herbicide_1_RR[0:25,0] = Herbicide_data[0:25,3]
Herbicide_1_RR[0:25,1] = Herbicide_data[25:50,3]

In [ ]:
Herbicide_2_SS = np.zeros((25,2))
Herbicide_2_RS = np.zeros((25,2))
Herbicide_2_RR = np.zeros((25,2))

Herbicide_3_SS = np.zeros((25,2))
Herbicide_3_RS = np.zeros((25,2))
Herbicide_3_RR = np.zeros((25,2))

Herbicide_4_SS = np.zeros((25,2))
Herbicide_4_RS = np.zeros((25,2))
Herbicide_4_RR = np.zeros((25,2))


In [ ]:
nYears = General_nYears
nGeno = General_nGeno
nCohorts = General_nCohorts


Population_seedBank = np.zeros((nYears,nGeno,nCohorts))

#Lower seed bank density
Population_lowerBank = np.zeros((nYears,nGeno))


Population_immSeed = np.zeros((nYears,nGeno,nCohorts))

Population_emSeed = np.zeros((nYears,nGeno,nCohorts))


Population_germination = np.zeros((nYears,nGeno,nCohorts))



Population_herbicide1 = np.zeros((nYears,nGeno,nCohorts))
Population_herbicide2 = np.zeros((nYears,nGeno,nCohorts))
Population_herbicide3 = np.zeros((nYears,nGeno,nCohorts))
Population_herbicide4 = np.zeros((nYears,nGeno,nCohorts))


Population_mature = np.zeros((nYears,nGeno,nCohorts))


Population_hand = np.zeros((nYears,nGeno,nCohorts))


Population_immPol = np.zeros((nYears,nGeno,nCohorts))


Population_emPol = np.zeros((nYears,nGeno,nCohorts))


Population_newSeed = np.zeros((nYears,nGeno,nCohorts))


Population_mutated = np.zeros((nYears,nGeno,nCohorts))


Population_predation = np.zeros((nYears,nGeno,nCohorts))


Population_winter = np.zeros((nYears,nGeno))


Population_Res = np.zeros((1, 3))



Population_seedBank(1,:,1) = Params.General.UpperSeedBank
Population_lowerBank(1,:) = Params.General.LowerSeedBank

SyntaxError: invalid syntax (<ipython-input-40-3c4456759482>, line 55)

In [ ]:
import math
def generateKey(nLoci):

    nGeno = math.pow(3,nLoci)


    key = np.zeros(nLoci,nGeno)

    tmp = np.zeros(nLoci,1)

    for k in range(len(nGeno)):
        key[:,k] = tmp
        tmp = odometer(tmp,2)

    key_text = 0

    key_text = cell(1,nGeno)

    for k in range(len(nGeno)):
        tmp = ''
        for j in range(len(nLoci)):
            locus_string = str(j)
            if key(j,k) == 0:
                tmp = tmp+'S'+locus_string+'S'+locus_string
            elif key(j,k) == 1:
                tmp = tmp+'R'+locus_string+'S'+locus_string
            else:
                tmp = tmp+'R'+locus_string+'R'+locus_string


        key_text{k} = tmp


    
function odo = odometer(odo,base_num)
%% Odometer function
%This function counts using a user specified number base. A digit takes
%on integer values between 0 and base_num. This function increments the
%right-most digit, unless the right-most digit is at the maximum value,
%in which case it is set to the first element of base_num and the next
%element is incremented. For example: odo = [0,0,1] will become odo =
%[0,0,2]. A second function call on odo will produce odo = [0,1,0].
%This works like a standard car odometer, but with an arbitrary number
%system.

N = length(base_num);       %number of possible elements
Ndigits = length(odo);      %number of elements in odo

j = 1;
while j <= Ndigits
    if odo(j) < base_num
        odo(j) = odo(j)+1;
        break
    end
    odo(j) = 0;
    j = j + 1;
end
end

function populationFraction_outCross = matingEquilibrium(p,selfCoeff,key)
q = 1-p;
nLoci = size(key,1);
nGeno = size(key,2);
populationFraction_outCross = ones(1,nGeno);
%     populationFraction_selfing = ones(1,nGeno);

for j = 1:nLoci
    for k = 1:nGeno
        if key(j,k) == 2
            populationFraction_outCross(k) = populationFraction_outCross(k)*p(j)^2;
        elseif key(j,k) == 1
            populationFraction_outCross(k) = populationFraction_outCross(k)*2*p(j)*q(j);
        else
            populationFraction_outCross(k) = populationFraction_outCross(k)*q(j)^2;
        end
    end
end
end